In [16]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import regularizers
from keras import layers, models
from keras.applications import VGG16
from keras import Input
from keras.models import Model
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint
import os
from glob import glob
from PIL import Image
import numpy as np

In [17]:
data_dir = "./data"
categories = []

In [18]:
for i in range(1, 40):
    categories.append(f"{i}")

In [19]:
print(categories)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39']


In [20]:
num_classes = len(categories)

image_w = 300
image_h = 400

pixels = image_w * image_h * 3

x = []
y = []

for idx, category in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idx] = 1

    image_dir = data_dir + "/" + category
    files = glob(image_dir + "/*.jpg")

    for i,f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        x.append(data)
        y.append(label)

        if i % 1000 == 0:
            print(category, " : ", f)

1  :  ./data/1\1_0_0_20161219140623097.jpg.chip.jpg
1  :  ./data/1\1_1_3_20161220143211359.jpg.chip.jpg
2  :  ./data/2\2_0_0_20161219154008997.jpg.chip.jpg
3  :  ./data/3\3_0_0_20161219154520213.jpg.chip.jpg
4  :  ./data/4\4_0_0_20161219192808843.jpg.chip.jpg
5  :  ./data/5\5_0_0_20170103205053570.jpg.chip.jpg
6  :  ./data/6\6_0_0_20170110213109002.jpg.chip.jpg
7  :  ./data/7\7_0_0_20161219201514284.jpg.chip.jpg
8  :  ./data/8\8_0_0_20170103200436055.jpg.chip.jpg
9  :  ./data/9\9_0_0_20170102235106821.jpg.chip.jpg
10  :  ./data/10\10_0_0_20161220222308131.jpg.chip.jpg
11  :  ./data/11\11_0_0_20170103200509559.jpg.chip.jpg
12  :  ./data/12\12_0_0_20170103200900511.jpg.chip.jpg
13  :  ./data/13\13_0_0_20170103200413990.jpg.chip.jpg
14  :  ./data/14\14_0_0_20170102234323550.jpg.chip.jpg
15  :  ./data/15\15_0_0_20170102234355667.jpg.chip.jpg
16  :  ./data/16\16_0_0_20170102234641453.jpg.chip.jpg
17  :  ./data/17\17_0_0_20170103201439825.jpg.chip.jpg
18  :  ./data/18\18_0_0_2017010320130800

In [21]:
x = np.array(x)
y = np.array(y)

MemoryError: Unable to allocate 5.52 GiB for an array with shape (16472, 400, 300, 3) and data type uint8

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y)
xy = (x_train, x_test, y_train, y_test)

x_train = x_train.astype(float) / 255
x_test = x_test.astype(float) / 255

MemoryError: Unable to allocate 47.6 GiB for an array with shape (17763, 400, 300, 3) and data type float64

In [ ]:
input_tensor = Input(shape=(400, 300, 3), dtype='float32', name='input')

#vgg16 모델 불러오기
pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(400, 300, 3))
pre_trained_vgg.trainable = False
pre_trained_vgg.summary()

#vgg16 밑에 레이어 추가
additional_model = models.Sequential()
additional_model.add(pre_trained_vgg)
additional_model.add(layers.Flatten())
additional_model.add(layers.Dense(4096, kernel_regularizer = regularizers.l1_l2
                                  (l1=0.001,l2=0.001),activation='relu'))
additional_model.add(layers.Dropout(0.5))
additional_model.add(layers.Dense(2048, kernel_regularizer = regularizers.l1_l2
                                  (l1=0.001,l2=0.001),activation='relu'))
additional_model.add(layers.Dropout(0.5))
additional_model.add(layers.Dense(1024, kernel_regularizer = regularizers.l1_l2
                                  (l1=0.001,l2=0.001),activation='relu'))
additional_model.add(layers.Dropout(0.5))
additional_model.add(layers.Dense(12, activation='softmax'))

additional_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
additional_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 222, 222, 128)     3584      
                                                                 
 conv2d_13 (Conv2D)          (None, 220, 220, 128)     147584    
                                                                 
 conv2d_14 (Conv2D)          (None, 218, 218, 128)     147584    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 108, 108, 128)    0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 106, 106, 128)     147584    
                                                           

In [ ]:
additional_model.fit(x_train, y_train, 
                    batch_size=50, 
                    epochs=500, 
                    validation_data=(x_test, y_test))

Physical devices cannot be modified after being initialized


In [ ]:
model_json = additional_model.to_json()
with open("model.json", "w") as json_file : 
    json_file.write(model_json)

(16579, 224, 224, 3) 4
(16579, 100) 2
